# 第 8 章：模組與套件

本章節詳細說明 Python 的模組系統、套件結構、匯入方式，以及常用的標準函式庫。

---

## 8.1 匯入方式

### JavaScript vs Python 匯入對照

| JavaScript | Python | 說明 |
|------------|--------|------|
| `import x from 'mod'` | `from mod import x` | 匯入特定項目 |
| `import { a, b } from 'mod'` | `from mod import a, b` | 匯入多個項目 |
| `import * as mod from 'mod'` | `import mod` | 匯入整個模組 |
| `import 'mod'` | `import mod` | 匯入模組（副作用） |
| `export default x` | 無直接對應 | Python 無預設匯出 |
| `export { x, y }` | 自動匯出所有公開成員 | Python 不需要 export |

### import 語句

In [ ]:
# 匯入整個模組
import math

# 使用時需要加上模組名稱
print(math.pi)          # 3.141592653589793
print(math.sqrt(16))    # 4.0
print(math.sin(0))      # 0.0

In [ ]:
# 匯入多個模組
import os
import sys
import json

print("os 模組:", os.name)
print("sys 版本:", sys.version_info[:2])

### from ... import 語句

In [ ]:
# 匯入特定項目
from math import pi, sqrt, sin

# 使用時不需要模組名稱
print(pi)          # 3.141592653589793
print(sqrt(16))    # 4.0
print(sin(0))      # 0.0

#### 為什麼不推薦 `from module import *`

```python
# 匯入所有公開成員（不推薦）
from math import *
```

不推薦的原因：
1. 可能覆蓋現有變數
2. 難以追蹤名稱來源
3. 增加命名衝突風險

例外情況：
- 在互動式環境中方便測試
- 模組明確設計為 `import *`（如 `tkinter`）

### 匯入別名（as）

In [ ]:
# 模組別名 - 常見慣例
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt

# 示範：使用別名
import math as m

print(m.pi)
print(m.sqrt(25))

In [ ]:
# 項目別名
from datetime import datetime as dt
from collections import defaultdict as dd

now = dt.now()
print("現在時間:", now)

counts = dd(int)
counts["a"] += 1
print("計數:", dict(counts))

#### JavaScript 對照

```javascript
// JavaScript
import { helper as helperA } from './module_a';
import * as np from 'numpy';  // 類似 Python 的 import numpy as np
```

### 條件匯入

In [ ]:
# 根據條件匯入不同模組
import sys

print(f"Python 版本: {sys.version_info.major}.{sys.version_info.minor}")

if sys.version_info >= (3, 11):
    print("可以使用 Python 3.11+ 的 tomllib")
else:
    print("需要使用 tomli 套件")

In [ ]:
# 匯入可選模組
try:
    import numpy as np
    HAS_NUMPY = True
    print("NumPy 已安裝")
except ImportError:
    HAS_NUMPY = False
    print("NumPy 未安裝")

def process_array(data):
    if HAS_NUMPY:
        return np.array(data)
    return list(data)

result = process_array([1, 2, 3])
print(f"結果類型: {type(result).__name__}")

In [ ]:
# 延遲匯入（改善啟動時間）
def heavy_computation():
    import json  # 只在需要時才匯入
    data = {"key": "value"}
    return json.dumps(data)

# 模組只在函式被呼叫時才載入
print(heavy_computation())

### 匯入的搜尋路徑

In [ ]:
import sys

# 查看模組搜尋路徑
print("模組搜尋路徑:")
for i, path in enumerate(sys.path[:5]):
    print(f"  {i}: {path}")
print("  ...（更多路徑）")

In [ ]:
# 查看模組位置
import os
print(f"os 模組位置: {os.__file__}")

# 查看模組的所有屬性
print(f"\nos 模組的部分屬性:")
print([attr for attr in dir(os) if not attr.startswith('_')][:10])

---

## 8.2 模組結構

### 單一檔案模組

```python
# mymodule.py

"""
這是模組的 docstring。
說明模組的用途和使用方式。
"""

# 模組級別的變數
VERSION = "1.0.0"
_private_var = "內部使用"

# 模組級別的函式
def greet(name):
    """問候函式"""
    return f"Hello, {name}!"

def _private_function():
    """內部函式"""
    pass

# 類別
class MyClass:
    """範例類別"""
    pass

# __all__ 定義 import * 時匯出的名稱
__all__ = ["VERSION", "greet", "MyClass"]
```

### 使用模組

```python
# main.py

# 匯入模組
import mymodule

print(mymodule.VERSION)        # "1.0.0"
print(mymodule.greet("Alice")) # "Hello, Alice!"

# from import
from mymodule import greet, MyClass

# import * 只會匯入 __all__ 中的項目
from mymodule import *
print(VERSION)      # "1.0.0"
print(greet)        # <function greet>
# print(_private_var)  # NameError（不在 __all__ 中）
```

### 套件結構

套件是包含 `__init__.py` 的目錄：

```
my_package/
├── __init__.py          # 套件初始化檔案
├── module_a.py          # 子模組 A
├── module_b.py          # 子模組 B
├── subpackage/          # 子套件
│   ├── __init__.py
│   └── module_c.py
└── utils/               # 工具子套件
    ├── __init__.py
    ├── helpers.py
    └── validators.py
```

### `__init__.py` 檔案

```python
# my_package/__init__.py

"""
My Package - 套件說明

這個套件提供 XXX 功能。
"""

# 套件版本
__version__ = "1.0.0"

# 從子模組匯入，讓使用者可以直接從套件匯入
from .module_a import function_a, ClassA
from .module_b import function_b

# 定義公開 API
__all__ = [
    "function_a",
    "function_b",
    "ClassA",
    "__version__",
]

# 套件初始化程式碼
print(f"載入 my_package v{__version__}")
```

### 匯入子模組的方式

```python
# 方式 1：匯入子模組
import my_package.module_a
my_package.module_a.function_a()

# 方式 2：from 匯入
from my_package import module_a
module_a.function_a()

# 方式 3：直接匯入函式
from my_package.module_a import function_a
function_a()

# 方式 4：匯入子套件
from my_package.subpackage import function_c
function_c()

# 方式 5：深層匯入
from my_package.subpackage.module_c import function_c
```

---

## 8.3 相對匯入與絕對匯入

### 絕對匯入

```python
# 使用完整的套件路徑
from my_package.module_a import function_a
from my_package.subpackage.module_c import function_c
from my_package.utils.helpers import helper_function
```

### 相對匯入

在套件內部使用，以 `.` 開頭：

```python
# my_package/module_b.py

# . 代表當前套件
from . import module_a
from .module_a import function_a

# .. 代表父套件
from .. import other_package
from ..other_package import something

# 子套件
from .subpackage import function_c
from .utils.helpers import helper_function
```

### 相對匯入的規則

```python
# my_package/subpackage/module_c.py

# . 當前目錄（subpackage）
from . import another_module      # subpackage/another_module.py
from .another_module import func

# .. 上一層目錄（my_package）
from .. import module_a           # my_package/module_a.py
from ..module_a import function_a

# ... 上兩層目錄
from ... import top_level_module
```

**注意**：相對匯入只能在套件內使用
```bash
python my_package/module_b.py  # 會失敗
python -m my_package.module_b  # 可以
```

### `__name__` 與 `__main__`

In [ ]:
# 示範 __name__ 的值
print(f"目前 __name__ = {__name__}")

# 在 Jupyter notebook 中，__name__ 通常是 "__main__"
# 當模組被匯入時，__name__ 是模組名稱

#### 典型用法

```python
# my_module.py

def main():
    print("主程式執行")

def helper():
    print("輔助函式")

# 只在直接執行時運行
if __name__ == "__main__":
    main()

# 當模組被匯入時，__name__ 是模組名稱
# 當模組直接執行時，__name__ 是 "__main__"
```

### 套件的 `__main__.py`

```
my_package/
├── __init__.py
├── __main__.py      # python -m my_package 時執行
└── core.py
```

```python
# my_package/__main__.py

"""
讓套件可以用 python -m my_package 執行
"""

from .core import main

if __name__ == "__main__":
    main()
```

```bash
# 執行套件
python -m my_package
```

---

## 8.4 常用標準函式庫

### os - 作業系統介面

In [ ]:
import os

# 環境變數
print("HOME:", os.environ.get("HOME"))

# 當前工作目錄
print("目前目錄:", os.getcwd())

In [ ]:
# 目錄操作示範
import os

# mkdir - 建立單一目錄
# os.mkdir("new_dir")

# makedirs - 建立多層目錄（類似 mkdir -p）
# os.makedirs("path/to/new/dir", exist_ok=True)

# rmdir - 刪除空目錄
# os.rmdir("empty_dir")

# 列出目錄內容
print("目前目錄內容:")
for item in os.listdir(".")[:5]:
    print(f"  {item}")

In [ ]:
# os.scandir - 更高效的目錄遍歷
import os

print("使用 scandir:")
with os.scandir(".") as entries:
    for entry in list(entries)[:5]:
        print(f"  {entry.name}: 檔案={entry.is_file()}, 目錄={entry.is_dir()}")

### os.path - 路徑操作

In [ ]:
import os.path

# 路徑組合
path = os.path.join("dir", "subdir", "file.txt")
print(f"組合路徑: {path}")

# 分解路徑
sample_path = "/path/to/file.txt"
print(f"dirname: {os.path.dirname(sample_path)}")
print(f"basename: {os.path.basename(sample_path)}")
print(f"split: {os.path.split(sample_path)}")

In [ ]:
import os.path

# 副檔名
print(f"splitext: {os.path.splitext('file.txt')}")

# 絕對路徑
print(f"絕對路徑: {os.path.abspath('file.txt')}")

# 檢查路徑
print(f"'.' 是否存在: {os.path.exists('.')}")
print(f"'.' 是否為目錄: {os.path.isdir('.')}")

### pathlib - 現代路徑操作（推薦）

In [ ]:
from pathlib import Path

# 建立 Path 物件
p = Path.cwd()  # 當前目錄
print(f"當前目錄: {p}")

home = Path.home()  # 家目錄
print(f"家目錄: {home}")

In [ ]:
from pathlib import Path

# 路徑組合（使用 / 運算子）- 這是 Python 特有的優雅語法！
config_path = Path.home() / ".config" / "myapp" / "config.json"
print(f"設定檔路徑: {config_path}")

# JavaScript 對照：path.join(os.homedir(), '.config', 'myapp', 'config.json')

In [ ]:
from pathlib import Path

# 路徑資訊
p = Path("/path/to/file.txt")

print(f"name: {p.name}")          # 檔案名稱
print(f"stem: {p.stem}")          # 不含副檔名的檔案名稱
print(f"suffix: {p.suffix}")      # 副檔名
print(f"parent: {p.parent}")      # 父目錄
print(f"parts: {p.parts}")        # 路徑各部分的元組

In [ ]:
from pathlib import Path

p = Path(".")

# 檢查
print(f"exists: {p.exists()}")
print(f"is_file: {p.is_file()}")
print(f"is_dir: {p.is_dir()}")
print(f"absolute: {p.absolute()}")

In [ ]:
from pathlib import Path

# 遍歷目錄
print("目錄內容:")
for item in list(Path(".").iterdir())[:5]:
    print(f"  {item}")

In [ ]:
from pathlib import Path

# Glob 模式匹配
print("Python 檔案:")
for py_file in list(Path(".").glob("*.py"))[:5]:
    print(f"  {py_file}")

# rglob - 遞迴搜尋
print("\n遞迴搜尋 .ipynb 檔案:")
for nb_file in list(Path(".").rglob("*.ipynb"))[:5]:
    print(f"  {nb_file}")

#### pathlib 讀寫檔案

```python
from pathlib import Path

p = Path("example.txt")

# 讀取文字
content = p.read_text(encoding="utf-8")

# 寫入文字
p.write_text("Hello", encoding="utf-8")

# 讀取二進位
data = p.read_bytes()

# 寫入二進位
p.write_bytes(b"binary data")
```

### sys - 系統特定參數

In [ ]:
import sys

# Python 版本
print(f"版本: {sys.version}")
print(f"版本資訊: {sys.version_info}")

# 版本檢查
if sys.version_info >= (3, 10):
    print("可以使用 Python 3.10+ 的功能（如 match-case）")

In [ ]:
import sys

# 命令列參數（在 notebook 中會不同）
print(f"argv: {sys.argv[:3]}...")

# 遞迴限制
print(f"遞迴限制: {sys.getrecursionlimit()}")

# 物件大小
print(f"list [1,2,3] 大小: {sys.getsizeof([1, 2, 3])} bytes")

#### sys 其他功能

```python
import sys

# 標準輸入輸出
sys.stdout.write("輸出到標準輸出\n")
sys.stderr.write("輸出到標準錯誤\n")

# 退出程式
sys.exit(0)  # 正常退出
sys.exit(1)  # 錯誤退出
sys.exit("錯誤訊息")

# 修改遞迴限制
sys.setrecursionlimit(2000)
```

### json - JSON 編解碼

In [ ]:
import json

# Python 物件轉 JSON 字串
data = {
    "name": "Alice",
    "age": 30,
    "cities": ["Taipei", "Tokyo"],
    "active": True,
    "score": None
}

json_str = json.dumps(data)
print("壓縮格式:")
print(json_str)

In [ ]:
import json

data = {"name": "小明", "age": 25, "cities": ["台北", "高雄"]}

# 格式化輸出
json_str = json.dumps(data, indent=2, ensure_ascii=False)
print("格式化輸出:")
print(json_str)

In [ ]:
import json

# JSON 字串轉 Python 物件
json_str = '{"name": "Bob", "age": 25}'
obj = json.loads(json_str)

print(f"名稱: {obj['name']}")
print(f"年齡: {obj['age']}")
print(f"型別: {type(obj)}")

#### 讀寫 JSON 檔案

```python
import json

# 寫入 JSON 檔案
with open("data.json", "w", encoding="utf-8") as f:
    json.dump(data, f, indent=2, ensure_ascii=False)

# 讀取 JSON 檔案
with open("data.json", "r", encoding="utf-8") as f:
    loaded_data = json.load(f)
```

In [ ]:
# 自訂編碼器
import json
from datetime import datetime

class CustomEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, datetime):
            return obj.isoformat()
        return super().default(obj)

data = {"timestamp": datetime.now(), "message": "測試"}
json_str = json.dumps(data, cls=CustomEncoder, ensure_ascii=False)
print(json_str)

### datetime - 日期時間

In [ ]:
from datetime import datetime, date, time, timedelta, timezone

# 當前時間
now = datetime.now()
utc_now = datetime.now(timezone.utc)

print(f"本地時間: {now}")
print(f"UTC 時間: {utc_now}")

In [ ]:
from datetime import datetime, date, time

# 建立特定日期時間
dt = datetime(2024, 1, 15, 10, 30, 45)
d = date(2024, 1, 15)
t = time(10, 30, 45)

print(f"datetime: {dt}")
print(f"date: {d}")
print(f"time: {t}")

In [ ]:
from datetime import datetime, date

# 從字串解析
dt = datetime.strptime("2024-01-15 10:30:45", "%Y-%m-%d %H:%M:%S")
print(f"解析結果: {dt}")

d = date.fromisoformat("2024-01-15")
print(f"ISO 格式解析: {d}")

In [ ]:
from datetime import datetime

now = datetime.now()

# 格式化為字串
print(f"strftime: {now.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"isoformat: {now.isoformat()}")

# 取得各部分
print(f"年/月/日: {now.year}/{now.month}/{now.day}")
print(f"時/分/秒: {now.hour}:{now.minute}:{now.second}")
print(f"星期幾: {now.weekday()} (0=Monday, 6=Sunday)")

In [ ]:
from datetime import datetime, timedelta

now = datetime.now()

# 時間計算
tomorrow = now + timedelta(days=1)
next_week = now + timedelta(weeks=1)
two_hours_later = now + timedelta(hours=2)

print(f"現在: {now}")
print(f"明天: {tomorrow}")
print(f"下週: {next_week}")
print(f"兩小時後: {two_hours_later}")

In [ ]:
from datetime import datetime

# 時間差
start = datetime(2024, 1, 1)
end = datetime(2024, 12, 31)
diff = end - start

print(f"2024 年共有 {diff.days} 天")
print(f"總秒數: {diff.total_seconds()}")

### re - 正規表達式

In [ ]:
import re

text = "Email: alice@example.com, Phone: 123-456-7890"

# 搜尋
match = re.search(r"\d{3}-\d{3}-\d{4}", text)
if match:
    print(f"找到電話: {match.group()}")

In [ ]:
import re

text = "alice@example.com, bob@test.org, charlie@domain.net"

# 找出所有匹配
emails = re.findall(r"[\w.-]+@[\w.-]+", text)
print(f"找到的 email: {emails}")

In [ ]:
import re

text = "Phone: 123-456-7890"

# 取代
new_text = re.sub(r"\d", "X", text)
print(f"原文: {text}")
print(f"取代後: {new_text}")

In [ ]:
import re

# 分割
parts = re.split(r"[,;]\s*", "a, b; c, d")
print(f"分割結果: {parts}")

In [ ]:
import re

# 編譯正規表達式（重複使用時效率較高）
pattern = re.compile(r"(\w+)@(\w+)\.(\w+)")

match = pattern.search("alice@example.com")
if match:
    print(f"完整匹配: {match.group()}")
    print(f"群組 1 (用戶): {match.group(1)}")
    print(f"群組 2 (網域): {match.group(2)}")
    print(f"所有群組: {match.groups()}")

In [ ]:
import re

# 命名群組
pattern = re.compile(r"(?P<user>\w+)@(?P<domain>\w+\.\w+)")

match = pattern.search("alice@example.com")
if match:
    print(f"user: {match.group('user')}")
    print(f"domain: {match.group('domain')}")
    print(f"groupdict: {match.groupdict()}")

### collections - 容器資料型別

In [ ]:
from collections import namedtuple

# namedtuple - 具名元組
Point = namedtuple("Point", ["x", "y"])
p = Point(10, 20)

print(f"Point: {p}")
print(f"x = {p.x}, y = {p.y}")

# 也可以用索引存取
print(f"p[0] = {p[0]}, p[1] = {p[1]}")

In [ ]:
from collections import deque

# deque - 雙端佇列（比 list 更高效的兩端操作）
dq = deque([1, 2, 3])

dq.append(4)       # 右邊加入
dq.appendleft(0)   # 左邊加入
print(f"加入後: {dq}")

dq.pop()           # 右邊移除
dq.popleft()       # 左邊移除
print(f"移除後: {dq}")

dq.rotate(1)       # 旋轉
print(f"旋轉後: {dq}")

In [ ]:
from collections import Counter

# Counter - 計數器
counter = Counter("abracadabra")
print(f"Counter: {counter}")
print(f"最常見 2 個: {counter.most_common(2)}")

# 也可以用於 list
words = ["apple", "banana", "apple", "cherry", "banana", "apple"]
word_count = Counter(words)
print(f"單字計數: {word_count}")

In [ ]:
from collections import defaultdict

# defaultdict - 預設值字典
# 類似 JavaScript 中需要 || 初始化的情況

dd = defaultdict(list)
dd["a"].append(1)
dd["a"].append(2)
dd["b"].append(3)
print(f"list defaultdict: {dict(dd)}")

# 計數器
dd = defaultdict(int)
for char in "hello":
    dd[char] += 1
print(f"int defaultdict: {dict(dd)}")

In [ ]:
from collections import ChainMap

# ChainMap - 鏈式字典
defaults = {"color": "blue", "size": "medium"}
custom = {"color": "red"}
combined = ChainMap(custom, defaults)

print(f"color: {combined['color']}")  # 從 custom
print(f"size: {combined['size']}")    # 從 defaults

### itertools - 迭代工具

In [ ]:
from itertools import count, cycle, islice

# count - 無限計數器
counter = count(10, 2)  # 從 10 開始，步進 2
print("count:", [next(counter) for _ in range(5)])

# cycle - 循環迭代
colors = cycle(["red", "green", "blue"])
print("cycle:", [next(colors) for _ in range(5)])

In [ ]:
from itertools import chain, islice

# chain - 串接多個迭代器
combined = chain([1, 2], [3, 4], [5, 6])
print(f"chain: {list(combined)}")

# islice - 切片迭代器
result = islice(range(100), 5, 15, 2)  # 從索引 5 到 15，步進 2
print(f"islice: {list(result)}")

In [ ]:
from itertools import groupby

# groupby - 分組
data = [("a", 1), ("a", 2), ("b", 3), ("b", 4)]

print("groupby 結果:")
for key, group in groupby(data, key=lambda x: x[0]):
    print(f"  {key}: {list(group)}")

In [ ]:
from itertools import permutations, combinations, product

# permutations - 排列
print(f"permutations: {list(permutations([1, 2, 3], 2))}")

# combinations - 組合
print(f"combinations: {list(combinations([1, 2, 3], 2))}")

# product - 笛卡爾積
print(f"product: {list(product([1, 2], ['a', 'b']))}")

### functools - 函式工具

In [ ]:
from functools import partial

# partial - 部分應用（類似 JavaScript 的 bind）
def power(base, exponent):
    return base ** exponent

square = partial(power, exponent=2)
cube = partial(power, exponent=3)

print(f"square(5) = {square(5)}")
print(f"cube(5) = {cube(5)}")

In [ ]:
from functools import reduce

# reduce - 累積運算（類似 JavaScript 的 Array.reduce）
result = reduce(lambda acc, x: acc + x, [1, 2, 3, 4, 5], 0)
print(f"reduce 加總: {result}")

# 計算階乘
factorial = reduce(lambda acc, x: acc * x, range(1, 6), 1)
print(f"5! = {factorial}")

In [ ]:
from functools import lru_cache

# lru_cache - 快取裝飾器（記憶化）
@lru_cache(maxsize=128)
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 1) + fibonacci(n - 2)

# 沒有快取的話，計算 fib(35) 會非常慢
print(f"fibonacci(35) = {fibonacci(35)}")
print(f"快取資訊: {fibonacci.cache_info()}")

In [ ]:
from functools import wraps

# wraps - 保留函式資訊的裝飾器
def my_decorator(func):
    @wraps(func)  # 保留原函式的名稱、docstring 等
    def wrapper(*args, **kwargs):
        print(f"呼叫 {func.__name__}")
        return func(*args, **kwargs)
    return wrapper

@my_decorator
def greet(name):
    """問候函式"""
    return f"Hello, {name}!"

print(greet("Alice"))
print(f"函式名稱: {greet.__name__}")
print(f"docstring: {greet.__doc__}")

### subprocess - 子程序管理

In [ ]:
import subprocess

# 執行命令並取得輸出
result = subprocess.run(
    ["echo", "Hello from subprocess"],
    capture_output=True,
    text=True
)

print(f"stdout: {result.stdout}")
print(f"returncode: {result.returncode}")

In [ ]:
import subprocess

# 執行 Python 命令
result = subprocess.run(
    ["python3", "-c", "print('Hello from Python subprocess')"],
    capture_output=True,
    text=True
)

print(f"輸出: {result.stdout}")

#### subprocess 進階用法

```python
import subprocess

# 執行命令並檢查錯誤
try:
    subprocess.run(["false"], check=True)
except subprocess.CalledProcessError as e:
    print(f"命令失敗，返回碼：{e.returncode}")

# 傳遞輸入
result = subprocess.run(
    ["grep", "hello"],
    input="hello world\nfoo bar\nhello again",
    capture_output=True,
    text=True
)

# 使用 shell（注意安全性）
result = subprocess.run(
    "ls -la | head -5",
    shell=True,
    capture_output=True,
    text=True
)
```

### logging - 日誌記錄

In [ ]:
import logging

# 基本設定
logging.basicConfig(
    level=logging.DEBUG,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

# 取得 logger
logger = logging.getLogger(__name__)

# 記錄訊息
logger.debug("除錯訊息")
logger.info("一般資訊")
logger.warning("警告訊息")
logger.error("錯誤訊息")

In [ ]:
import logging

logger = logging.getLogger("demo")

# 例外記錄
try:
    1 / 0
except Exception:
    logger.exception("發生例外")

#### logging 進階設定

```python
import logging

# 建立 logger
logger = logging.getLogger("myapp")
logger.setLevel(logging.DEBUG)

# 處理器（Handler）
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)

file_handler = logging.FileHandler("app.log")
file_handler.setLevel(logging.DEBUG)

# 格式器（Formatter）
formatter = logging.Formatter(
    "%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)
console_handler.setFormatter(formatter)
file_handler.setFormatter(formatter)

# 加入處理器
logger.addHandler(console_handler)
logger.addHandler(file_handler)
```

### argparse - 命令列參數解析

```python
import argparse

# 建立解析器
parser = argparse.ArgumentParser(
    description="程式說明"
)

# 位置參數
parser.add_argument("filename", help="輸入檔案")

# 選項參數
parser.add_argument("-o", "--output", help="輸出檔案")
parser.add_argument("-v", "--verbose", action="store_true", help="詳細輸出")
parser.add_argument("-n", "--number", type=int, default=10, help="數量")
parser.add_argument("--mode", choices=["fast", "slow"], default="fast")

# 解析參數
args = parser.parse_args()

print(args.filename)
print(args.output)
print(args.verbose)
print(args.number)

# 使用範例
# python script.py input.txt -o output.txt -v -n 20
```

---

## 8.5 常用第三方套件

### 套件管理

```bash
# pip - 套件安裝工具
pip install requests
pip install numpy pandas
pip install package==1.0.0  # 指定版本
pip install package>=1.0.0  # 最低版本

# 升級套件
pip install --upgrade package

# 移除套件
pip uninstall package

# 列出已安裝套件
pip list
pip freeze > requirements.txt

# 從 requirements.txt 安裝
pip install -r requirements.txt
```

### 虛擬環境

```bash
# 建立虛擬環境
python -m venv venv

# 啟用（macOS/Linux）
source venv/bin/activate

# 啟用（Windows）
venv\Scripts\activate

# 停用
deactivate

# 在虛擬環境中安裝套件
pip install requests
```

**JavaScript 對照**：類似 Node.js 的 `node_modules`，但 Python 的虛擬環境是整個 Python 環境的隔離副本。

### 常用第三方套件列表

| 用途 | 套件 | 說明 |
|------|------|------|
| HTTP 請求 | `requests` | 簡單的 HTTP 函式庫 |
| HTTP 客戶端 | `httpx` | 支援 async 的 HTTP 函式庫 |
| Web 框架 | `Flask` | 輕量級 Web 框架 |
| Web 框架 | `FastAPI` | 現代 async Web 框架 |
| Web 框架 | `Django` | 全功能 Web 框架 |
| 測試 | `pytest` | 測試框架 |
| 程式碼格式化 | `black` | 程式碼格式化工具 |
| Linting | `ruff` | 快速的 linter |
| 型別檢查 | `mypy` | 靜態型別檢查 |
| 資料處理 | `pandas` | 資料分析 |
| 數值計算 | `numpy` | 數值計算 |
| ORM | `SQLAlchemy` | 資料庫 ORM |
| 環境變數 | `python-dotenv` | 載入 .env 檔案 |

---

## 練習題

### 練習 1：使用 pathlib 列出檔案

使用 `pathlib` 列出當前目錄下所有的 `.ipynb` 檔案，並印出它們的名稱和大小。

In [ ]:
# 練習 1：你的程式碼
from pathlib import Path

# 提示：使用 Path.glob() 和 stat().st_size


In [ ]:
# 練習 1：參考解答
from pathlib import Path

print("Notebook 檔案列表:")
for nb_file in sorted(Path(".").glob("*.ipynb")):
    size_kb = nb_file.stat().st_size / 1024
    print(f"  {nb_file.name}: {size_kb:.1f} KB")

### 練習 2：JSON 處理

建立一個函式，接收一個 Python 字典，將其轉換為格式化的 JSON 字串，並處理 `datetime` 類型。

In [ ]:
# 練習 2：你的程式碼
import json
from datetime import datetime

def to_json(data):
    """將字典轉換為 JSON 字串，支援 datetime"""
    # 提示：使用自訂 JSONEncoder
    pass

# 測試
test_data = {
    "name": "測試",
    "timestamp": datetime.now(),
    "values": [1, 2, 3]
}
# print(to_json(test_data))

In [ ]:
# 練習 2：參考解答
import json
from datetime import datetime

class DateTimeEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, datetime):
            return obj.isoformat()
        return super().default(obj)

def to_json(data):
    """將字典轉換為 JSON 字串，支援 datetime"""
    return json.dumps(
        data, 
        cls=DateTimeEncoder, 
        indent=2, 
        ensure_ascii=False
    )

# 測試
test_data = {
    "name": "測試",
    "timestamp": datetime.now(),
    "values": [1, 2, 3]
}
print(to_json(test_data))

### 練習 3：日期計算

寫一個函式，計算兩個日期之間的工作天數（排除週末）。

In [ ]:
# 練習 3：你的程式碼
from datetime import date, timedelta

def count_workdays(start_date, end_date):
    """計算兩個日期之間的工作天數（排除週末）"""
    # 提示：使用 weekday() 方法，0-4 是週一到週五
    pass

# 測試
# start = date(2024, 1, 1)
# end = date(2024, 1, 31)
# print(f"工作天數: {count_workdays(start, end)}")

In [ ]:
# 練習 3：參考解答
from datetime import date, timedelta

def count_workdays(start_date, end_date):
    """計算兩個日期之間的工作天數（排除週末）"""
    workdays = 0
    current = start_date
    
    while current <= end_date:
        # weekday(): 0-4 是週一到週五
        if current.weekday() < 5:
            workdays += 1
        current += timedelta(days=1)
    
    return workdays

# 測試
start = date(2024, 1, 1)
end = date(2024, 1, 31)
print(f"2024 年 1 月工作天數: {count_workdays(start, end)} 天")

### 練習 4：使用 collections

使用 `Counter` 和 `defaultdict` 分析一段文字，統計每個單字出現的次數，並按照首字母分組。

In [ ]:
# 練習 4：你的程式碼
from collections import Counter, defaultdict

text = "the quick brown fox jumps over the lazy dog the fox is quick"

# 1. 統計單字出現次數
# 2. 按首字母分組


In [ ]:
# 練習 4：參考解答
from collections import Counter, defaultdict

text = "the quick brown fox jumps over the lazy dog the fox is quick"
words = text.split()

# 1. 統計單字出現次數
word_counts = Counter(words)
print("單字出現次數:")
for word, count in word_counts.most_common(5):
    print(f"  {word}: {count}")

# 2. 按首字母分組
by_first_letter = defaultdict(list)
for word in set(words):  # 使用 set 避免重複
    by_first_letter[word[0]].append(word)

print("\n按首字母分組:")
for letter in sorted(by_first_letter.keys()):
    print(f"  {letter}: {by_first_letter[letter]}")

### 練習 5：正規表達式提取資料

從一段文字中提取所有的 URL 和 Email 地址。

In [ ]:
# 練習 5：你的程式碼
import re

text = """
請聯繫 support@example.com 或訪問 https://www.example.com/help
其他聯繫方式: admin@test.org, http://docs.test.org/guide
"""

# 提取 URLs 和 Emails


In [ ]:
# 練習 5：參考解答
import re

text = """
請聯繫 support@example.com 或訪問 https://www.example.com/help
其他聯繫方式: admin@test.org, http://docs.test.org/guide
"""

# Email 正規表達式
email_pattern = r"[\w.-]+@[\w.-]+\.[a-zA-Z]{2,}"
emails = re.findall(email_pattern, text)
print(f"找到的 Emails: {emails}")

# URL 正規表達式
url_pattern = r"https?://[\w.-]+[\w/.-]*"
urls = re.findall(url_pattern, text)
print(f"找到的 URLs: {urls}")

---

## 小結

### 匯入方式對照

| 方式 | 語法 | 使用情境 |
|------|------|----------|
| 匯入模組 | `import module` | 需要使用模組的多個功能 |
| 匯入特定項目 | `from module import item` | 只需要少數幾個功能 |
| 匯入並取別名 | `import module as m` | 模組名稱太長 |
| 相對匯入 | `from . import module` | 套件內部匯入 |

### 常用標準函式庫

| 類別 | 模組 |
|------|------|
| 檔案/路徑 | `os`, `pathlib`, `shutil` |
| 系統 | `sys`, `subprocess` |
| 資料格式 | `json`, `csv`, `xml` |
| 日期時間 | `datetime`, `time`, `calendar` |
| 文字處理 | `re`, `string`, `textwrap` |
| 資料結構 | `collections`, `heapq`, `bisect` |
| 函式工具 | `functools`, `itertools`, `operator` |
| 網路 | `urllib`, `http`, `socket` |
| 並行處理 | `threading`, `multiprocessing`, `asyncio` |
| 測試 | `unittest`, `doctest` |
| 日誌 | `logging` |
| 命令列 | `argparse` |

### JavaScript vs Python 匯入

```javascript
// JavaScript
import module from 'module';
import { a, b } from 'module';
import * as mod from 'module';
export default x;
export { a, b };
```

```python
# Python
import module
from module import a, b
import module as mod
# 無 export，所有公開成員自動可匯入
# 使用 __all__ 控制 import *
```

### 關鍵差異

| 特性 | JavaScript | Python |
|------|------------|--------|
| 預設匯出 | `export default` | 無直接對應 |
| 明確匯出 | `export { }` | 自動匯出公開成員 |
| 控制匯出 | 使用 `export` | 使用 `__all__` |
| 私有成員 | 無原生支援 | 底線前綴慣例 |
| 套件管理 | npm/yarn | pip/venv |
| 路徑組合 | `path.join()` | `pathlib` 的 `/` 運算子 |